In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import bs4
from io import StringIO
import boto3
import time
import json

In [21]:
s3 = boto3.resource('s3')
s3.create_bucket(Bucket='macss30123amazondata')

s3.Bucket(name='macss30123amazondata')

In [255]:
# Access our class IAM role, which allows Lambda
# to interact with other AWS resources

aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')

# Open our Zipped directory
with open('amazon_scraper.zip', 'rb') as f:
    lambda_zip = f.read()

try:
    # If function hasn't yet been created, create it
    response = aws_lambda.create_function(
        FunctionName='amazon_scraper',
        Runtime='python3.9',
        Role=role['Role']['Arn'],
        Handler='lambda_function.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=900
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip
    # file contents
    response = aws_lambda.update_function_code(
    FunctionName='amazon_scraper',
    ZipFile=lambda_zip
    )

In [256]:
sfn = boto3.client('stepfunctions')
response = sfn.list_state_machines()
print(response)

{'stateMachines': [{'stateMachineArn': 'arn:aws:states:us-east-1:834397012326:stateMachine:amazon_scraper', 'name': 'amazon_scraper', 'type': 'EXPRESS', 'creationDate': datetime.datetime(2022, 5, 30, 16, 47, 6, 984000, tzinfo=tzlocal())}], 'ResponseMetadata': {'RequestId': '6af8d5ff-e4dd-43be-b7e8-9b3ceb6bb70f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6af8d5ff-e4dd-43be-b7e8-9b3ceb6bb70f', 'date': 'Tue, 31 May 2022 03:42:30 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '180'}, 'RetryAttempts': 0}}


In [257]:
# Get arn for Step Function state machine
response = sfn.list_state_machines()
state_machine_arn = [sm['stateMachineArn'] 
                     for sm in response['stateMachines'] 
                     if sm['name'] == 'amazon_scraper'][0]

In [258]:
pages = list(range(1,50))
n = 10//10
page_batch = [{'page': pages[i:i + n]} for i in range(0, len(pages), n)]

In [260]:
# Spread ISBN batches across Lambda workers
start = time.time()
response = sfn.start_sync_execution(
    stateMachineArn=state_machine_arn,
    name='amazon_scraper',
    input=json.dumps(page_batch)
)
time_elapsed = time.time() - start
time_elapsed

123.1585590839386